In [ ]:
# def integral_image(self, original_image):
#         h, w = original_image.shape
#         integral = np.zeros((h+1, w+1))

#         for y in range(1, h+1):
#             for x in range(1, w+1):
#                 integral[y, x] = integral[y, x-1] + integral[y-1, x] - integral[y-1, x-1] + original_image[y-1, x-1]

#         return integral


In [ ]:
import cv2
imgFile = cv2.imread('dataset/trainset/faces/face00001.png',0)

cv2.imshow('dst_rt', imgFile)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
import pickle
from PIL import Image
import cv2
import os
import random

#creating training data
faces_dir="dataset/trainset/faces"
face_images=os.listdir(faces_dir)
face=[]

for x in face_images:
    im = cv2.imread(faces_dir+"/"+x,0)
    face.append((im,1))
    
non_faces_dir="dataset/trainset/non-faces"
non_face_images=os.listdir(non_faces_dir)
non_face=[]

for x in non_face_images:
    im = cv2.imread(non_faces_dir+"/"+x,0)
    non_face.append((im,0))  
    
input_data = face+non_face
random.shuffle(input_data)
file = open('training_data.pkl', 'wb') 
pickle.dump(input_data, file)
print("DONE")    

DONE


In [6]:
#creating test dataset
faces_dir_test="dataset/testset/faces"
face_images_test=os.listdir(faces_dir_test)
face_test=[]

for x in face_images_test:
    im = cv2.imread(faces_dir_test+"/"+x,0)
    face_test.append((im,1))
    
    
non_faces_dir_test="dataset/testset/non-faces"
non_face_images_test=os.listdir(non_faces_dir_test)
non_face_test=[]

for x in non_face_images_test:
    im = cv2.imread(non_faces_dir_test+"/"+x,0)
    non_face_test.append((im,0)) 
    
input_data_test = face_test+non_face_test
random.shuffle(input_data_test)
file = open('testing_data.pkl', 'wb') 
pickle.dump(input_data_test, file)
print("DONE")    
    

DONE


In [ ]:
"""
A Python implementation of the Viola-Jones ensemble classification method described in 
Viola, Paul, and Michael Jones. "Rapid object detection using a boosted cascade of simple features." Computer Vision and Pattern Recognition, 2001. CVPR 2001. Proceedings of the 2001 IEEE Computer Society Conference on. Vol. 1. IEEE, 2001.
Works in both Python2 and Python3
"""
import numpy as np
import math
import pickle
from sklearn.feature_selection import SelectPercentile, f_classif

def integral_image(image):
    int_image = np.zeros(image.shape)
    s = np.zeros(image.shape)
    for i in range(len(image)):
        for j in range(len(image[i])):
            s[i][j] = s[i][j-1] + image[i][j] if j-1 >= 0 else image[i][j]
            int_image[i][j] = int_image[i-1][j]+s[i][j] if i-1 >= 0 else s[i][j]
    return int_image


def calculate(int_image, i, j, x, y):
    if i>=19:
        print("i {} is out of bounds".format(i))
    if j>=19:
        print("j {} is out of bounds".format(j))
    return int_image[i+x][j+y] + int_image[i][j] - (int_image[i][j+y]+int_image[i+x][j])


class ViolaJones:
    def __init__(self, r):
        """
          Args:
            T: The number of weak classifiers which should be used
        """
        self.rounds = r
        #self.weight=[]
        self.feature_coord=[]
        self.alphas = []
        self.polarities=[]
        self.thresholds=[]
        self.classifiers = []
        self.classifier_index=[]

    def feature_extraction(self, training, im_w, im_h):
        """
        Trains the Viola Jones classifier on a set of images (numpy arrays of shape (m, n))
          Args:
            training: An array of tuples. The first element is the numpy array of shape (m, n) representing the image. The second element is its classification (1 or 0)
        """
        flag=0
        type_1 = type_2 = type_3 = type_4 = type_5 = 0
        
        self.weights = np.zeros(len(training)) #######
        
        y_data=[]
        p=0
        n=0
        image_height, image_width = im_h, im_w
        for i in range(len(training)): #########
            if training[i][1]==1:
                p+=1
            elif training[i][1]==0:
                n+=1
            y_data.append(training[i][1])    
                
        features=[]
        for q in range(len(training)): ###########
            self.weights[q]=1/(2*p) if training[q][1]==1 else 1/(2*n)
            intermediate_1 = integral_image(training[q][0])
            image_features = []
            for w in range(1, image_width+1):
                for h in range(1, image_height+1):
                    i = 0
                    while i + w  < image_width and i<9:
                        j = 0
                        while j + h < image_height and j<9:
                            current = calculate(intermediate_1, i, j, w, h)
                            #2 horizontal
                            if i+2*w < image_width:
                                horizontal_1 = calculate(intermediate_1, i+w, j, w, h)
                                image_features.append(horizontal_1-current)
                                if not flag:
                                    type_2+=1
                                    self.feature_coord.append(([[i+w, j, w, h]], [[i, j, w, h]]))
                                
                                
                            #2 vertical    
                            if j+2*h < image_height:    
                                vertical_1 = calculate(intermediate_1, i, j+h, w, h)
                                image_features.append(current-vertical_1)
                                if not flag:
                                    type_1+=1
                                    self.feature_coord.append(([[i, j, w, h]],[[i, j+h, w, h]]))
                    
                    
                            #4 features    
                            if i+2*w < image_width and j+2*h < image_height:    
                                last = calculate(intermediate_1, i+w, j+h, w, h) 
                                image_features.append((horizontal_1+vertical_1)-(current+last))
                                if not flag:
                                    type_5+=1
                                    self.feature_coord.append(([[i+w, j, w, h],[i, j+h, w, h]],[[i, j, w, h],[i+w, j+h, w, h]]))
                            
                                
                            #3 horizontal      
                            if i+3*w < image_width:    
                                horizontal_2 = calculate(intermediate_1, i+2*w, j, w, h)
                                image_features.append(horizontal_1-(horizontal_2+current))
                                if not flag:
                                    type_3+=1
                                    self.feature_coord.append(([[i+w, j, w, h]],[[i+2*w, j, w, h],[i,j,w,h]]))
                               
                                
                            #3 vertical    
                            if j+3*h < image_height:    
                                vertical_2 = calculate(intermediate_1, i, j+2*h, w, h)
                                image_features.append(vertical_1-(vertical_2+current))
                                if not flag:
                                    type_4+=1
                                    self.feature_coord.append(([[i, j+h, w, h]],[[i, j+2*h, w, h],[i,j,w,h]]))
                            

                            j += 1
                        i += 1
            features.append(image_features)
            if not flag:
                print("Type 1: {}, Type 2: {}, Type 3: {}, Type 4: {}, Type 5: {}".format(type_1, type_2, type_3, type_4, type_5))
            flag=1
            print("Processing {}/{}".format(q,len(training))) #########
            
        #features = map(list, zip(*features)) 
        xx = features
        print("Shape of the data in feature extraction, before transpose is {}".format(np.asarray(xx).shape))
        x_data = np.transpose(features)
        print("Shape of the data in feature extraction, after transpose is {}".format(x_data.shape))
        return x_data, y_data
    
    
    
    
    def training(self, x_data, y_data):
        
        for r in range(self.rounds):
            w_data=self.weights
            print("Round : {}/{}______________________________________".format(r+1,self.rounds))
            #normalize the weights
            w_data = w_data / np.linalg.norm(w_data)
            
            number_of_features = len(self.feature_coord)
            
            #for selection of classifier
            best_classifier, best_classifier_index, best_error, best_accuracy = None, None, float('inf'), None
            
            #calculating weight of positive samples and weight of negative samples
            total_pos=0
            total_neg=0
            for w, y in zip(w_data, y_data):
                if y == 1:
                    total_pos += w
                else:
                    total_neg += w
                    

            #iterating through all the features for every round and choosing the best feature for each round
            for feature_index, feature in enumerate(x_data):
                feature=feature.tolist()  
                feature_sorted = sorted(zip(w_data, feature, y_data), key=lambda x: x[1])
                
                pos_seen, neg_seen = 0, 0
                pos_weights, neg_weights = 0, 0
                min_error, best_feature, best_threshold, best_polarity = float('inf'), None, None, None
                
                for w, f, y in feature_sorted:
                    error = min(neg_weights + total_pos - pos_weights, pos_weights + total_neg - neg_weights)
                    if error < min_error:
                        min_error = error
                        best_feature = self.feature_coord[feature_index]
                        best_feature_index = feature_index 
                        best_threshold = f
                        best_polarity = 1 if pos_seen > neg_seen else -1

                    if y == 1:
                        pos_seen += 1
                        pos_weights += w
                    else:
                        neg_seen += 1
                        neg_weights += w
                      
                #to find if this classifier is better than the previous best classifier        
                error, accuracy = 0, []
                for ind,w in enumerate(w_data):
                    if best_polarity * x_data[best_feature_index][ind] < best_polarity * best_threshold:
                        tmp=1
                    else:
                        tmp=0
                    correctness = abs(tmp - y_data[ind])
                    accuracy.append(correctness)
                    error += w * correctness
                error = error / len(y_data)
                
                #the best classifier among all the classifiers
                if error < best_error:
                    best_classifier, best_classifier_index, best_error, best_accuracy = best_feature, best_feature_index, error, accuracy
                    
                    
            #calculating alpha and beta values for the round        
            beta = error / (1.0 - error)
            alpha = math.log(1.0/beta)
            
            #weight updation
            for i in range(len(best_accuracy)):
                w_data[i] = w_data[i] * (beta ** (1 - best_accuracy[i]))
                
            #storing the values found in the round    
            self.classifiers.append(best_classifier) 
            self.alphas.append(alpha)
            self.thresholds.append(best_threshold)
            self.polarities.append(best_polarity)
            #self.classifier_index.append(best_classifier_index)
            #print("In round {}/{}:".format(r, self.rounds))
            print("The chosen classifier is {} and it's index is {}".format(best_classifier, best_classifier_index))
            print("Polarity is {}, threshold is {}".format(best_polarity, best_threshold))
            self.weights=w_data
            
            
    def classify(self, image):
        """
        Classifies an image
          Args:
            image: A numpy 2D array of shape (m, n) representing the image
          Returns:
            1 if the image is positively classified and 0 otherwise
        """
        test_result = 0
        int_image = integral_image(image)
        for alpha, classifier, polarity, threshold in zip(self.alphas, self.classifiers, self.polarities, self.thresholds): #clf=feature index; alpha=alpha
            tmp_pos=0
            tmp_neg=0
            feature_extraction_test=0
            
            for pos_reg in classifier[0]:
                feature_extraction_test += calculate(int_image, pos_reg[0],pos_reg[1],pos_reg[2],pos_reg[3])
            for neg_reg in classifier[1]:
                feature_extraction_test-= calculate(int_image, neg_reg[0], neg_reg[1], neg_reg[2], neg_reg[3])
                
            prediction=0
            if polarity*feature_extraction_test < polarity*threshold:
                prediction=1
            test_result+= alpha * prediction
        return 1 if test_result>=0.5*sum(self.alphas) else 0        
        
                
                    
    def evaluate(self, test_data):
        num_positives = 0
        num_negatives = 0
        
        correct = 0
        false_positives=0
        false_negatives=0
        
        #all_negatives, all_positives = 0, 0
        #true_negatives, false_negatives = 0, 0
        #true_positives, false_positives = 0, 0
        #classification_time = 0

        for x, y in test_data: #######
            if y == 1:
                num_positives += 1
            else:
                num_negatives += 1

            #start = time.time()
            pred = self.classify(x)
            #classification_time += time.time() - start
            if pred == 1 and y == 0:
                false_positives += 1
            if pred == 0 and y == 1:
                false_negatives += 1

            correct += 1 if pred == y else 0

        print("False Positive Rate: %d/%d (%f)" % (false_positives, num_negatives, false_positives/num_negatives))
        print("False Negative Rate: %d/%d (%f)" % (false_negatives, num_positives, false_negatives/num_positives))
        print("Accuracy: %d/%d (%f)" % (correct, len(test_data), correct/len(test_data))) #######
        #print("Average Classification Time: %f" % (classification_time / len(data)))  

        
viola = ViolaJones(10)
print("Viola object is created")        
            
with open("training_data.pkl", 'rb') as f:
    training = pickle.load(f)
    print("Training Data is loaded")
    
    x_train_data, y_train_data = viola.feature_extraction(training, 19, 19) #result is stored in the return
    print("Feature extraction is over")
    
    viola.training(x_train_data, y_train_data) #result is stored in self variables
    print("Tarining is completed")
    
with open('testing_data.pkl', 'rb') as ff:
    testing = pickle.load(ff)
    viola.evaluate(testing) #result is printed
    
    
            
            

Viola object is created
Training Data is loaded
Type 1: 7686, Type 2: 7686, Type 3: 4914, Type 4: 4914, Type 5: 3721
Processing 0/2499
Processing 1/2499
Processing 2/2499
Processing 3/2499
Processing 4/2499
Processing 5/2499
Processing 6/2499
Processing 7/2499
Processing 8/2499
Processing 9/2499
Processing 10/2499
Processing 11/2499
Processing 12/2499
Processing 13/2499
Processing 14/2499
Processing 15/2499
Processing 16/2499
Processing 17/2499
Processing 18/2499
Processing 19/2499
Processing 20/2499
Processing 21/2499
Processing 22/2499
Processing 23/2499
Processing 24/2499
Processing 25/2499
Processing 26/2499
Processing 27/2499
Processing 28/2499
Processing 29/2499
Processing 30/2499
Processing 31/2499
Processing 32/2499
Processing 33/2499
Processing 34/2499
Processing 35/2499
Processing 36/2499
Processing 37/2499
Processing 38/2499
Processing 39/2499
Processing 40/2499
Processing 41/2499
Processing 42/2499
Processing 43/2499
Processing 44/2499
Processing 45/2499
Processing 46/2499


Processing 411/2499
Processing 412/2499
Processing 413/2499
Processing 414/2499
Processing 415/2499
Processing 416/2499
Processing 417/2499
Processing 418/2499
Processing 419/2499
Processing 420/2499
Processing 421/2499
Processing 422/2499
Processing 423/2499
Processing 424/2499
Processing 425/2499
Processing 426/2499
Processing 427/2499
Processing 428/2499
Processing 429/2499
Processing 430/2499
Processing 431/2499
Processing 432/2499
Processing 433/2499
Processing 434/2499
Processing 435/2499
Processing 436/2499
Processing 437/2499
Processing 438/2499
Processing 439/2499
Processing 440/2499
Processing 441/2499
Processing 442/2499
Processing 443/2499
Processing 444/2499
Processing 445/2499
Processing 446/2499
Processing 447/2499
Processing 448/2499
Processing 449/2499
Processing 450/2499
Processing 451/2499
Processing 452/2499
Processing 453/2499
Processing 454/2499
Processing 455/2499
Processing 456/2499
Processing 457/2499
Processing 458/2499
Processing 459/2499
Processing 460/2499


Processing 824/2499
Processing 825/2499
Processing 826/2499
Processing 827/2499
Processing 828/2499
Processing 829/2499
Processing 830/2499
Processing 831/2499
Processing 832/2499
Processing 833/2499
Processing 834/2499
Processing 835/2499
Processing 836/2499
Processing 837/2499
Processing 838/2499
Processing 839/2499
Processing 840/2499
Processing 841/2499
Processing 842/2499
Processing 843/2499
Processing 844/2499
Processing 845/2499
Processing 846/2499
Processing 847/2499
Processing 848/2499
Processing 849/2499
Processing 850/2499
Processing 851/2499
Processing 852/2499
Processing 853/2499
Processing 854/2499
Processing 855/2499
Processing 856/2499
Processing 857/2499
Processing 858/2499
Processing 859/2499
Processing 860/2499
Processing 861/2499
Processing 862/2499
Processing 863/2499
Processing 864/2499
Processing 865/2499
Processing 866/2499
Processing 867/2499
Processing 868/2499
Processing 869/2499
Processing 870/2499
Processing 871/2499
Processing 872/2499
Processing 873/2499


Processing 1223/2499
Processing 1224/2499
Processing 1225/2499
Processing 1226/2499
Processing 1227/2499
Processing 1228/2499
Processing 1229/2499
Processing 1230/2499
Processing 1231/2499
Processing 1232/2499
Processing 1233/2499
Processing 1234/2499
Processing 1235/2499
Processing 1236/2499
Processing 1237/2499
Processing 1238/2499
Processing 1239/2499
Processing 1240/2499
Processing 1241/2499
Processing 1242/2499
Processing 1243/2499
Processing 1244/2499
Processing 1245/2499
Processing 1246/2499
Processing 1247/2499
Processing 1248/2499
Processing 1249/2499
Processing 1250/2499
Processing 1251/2499
Processing 1252/2499
Processing 1253/2499
Processing 1254/2499
Processing 1255/2499
Processing 1256/2499
Processing 1257/2499
Processing 1258/2499
Processing 1259/2499
Processing 1260/2499
Processing 1261/2499
Processing 1262/2499
Processing 1263/2499
Processing 1264/2499
Processing 1265/2499
Processing 1266/2499
Processing 1267/2499
Processing 1268/2499
Processing 1269/2499
Processing 12

Processing 1616/2499
Processing 1617/2499
Processing 1618/2499
Processing 1619/2499
Processing 1620/2499
Processing 1621/2499
Processing 1622/2499
Processing 1623/2499
Processing 1624/2499
Processing 1625/2499
Processing 1626/2499
Processing 1627/2499
Processing 1628/2499
Processing 1629/2499
Processing 1630/2499
Processing 1631/2499
Processing 1632/2499
Processing 1633/2499
Processing 1634/2499
Processing 1635/2499
Processing 1636/2499
Processing 1637/2499
Processing 1638/2499
Processing 1639/2499
Processing 1640/2499
Processing 1641/2499
Processing 1642/2499
Processing 1643/2499
Processing 1644/2499
Processing 1645/2499
Processing 1646/2499
Processing 1647/2499
Processing 1648/2499
Processing 1649/2499
Processing 1650/2499
Processing 1651/2499
Processing 1652/2499
Processing 1653/2499
Processing 1654/2499
Processing 1655/2499
Processing 1656/2499
Processing 1657/2499
Processing 1658/2499
Processing 1659/2499
Processing 1660/2499
Processing 1661/2499
Processing 1662/2499
Processing 16

Processing 2010/2499
Processing 2011/2499
Processing 2012/2499
Processing 2013/2499
Processing 2014/2499
Processing 2015/2499
Processing 2016/2499
Processing 2017/2499
Processing 2018/2499
Processing 2019/2499
Processing 2020/2499
Processing 2021/2499
Processing 2022/2499
Processing 2023/2499
Processing 2024/2499
Processing 2025/2499
Processing 2026/2499
Processing 2027/2499
Processing 2028/2499
Processing 2029/2499
Processing 2030/2499
Processing 2031/2499
Processing 2032/2499
Processing 2033/2499
Processing 2034/2499
Processing 2035/2499
Processing 2036/2499
Processing 2037/2499
Processing 2038/2499
Processing 2039/2499
Processing 2040/2499
Processing 2041/2499
Processing 2042/2499
Processing 2043/2499
Processing 2044/2499
Processing 2045/2499
Processing 2046/2499
Processing 2047/2499
Processing 2048/2499
Processing 2049/2499
Processing 2050/2499
Processing 2051/2499
Processing 2052/2499
Processing 2053/2499
Processing 2054/2499
Processing 2055/2499
Processing 2056/2499
Processing 20

Processing 2403/2499
Processing 2404/2499
Processing 2405/2499
Processing 2406/2499
Processing 2407/2499
Processing 2408/2499
Processing 2409/2499
Processing 2410/2499
Processing 2411/2499
Processing 2412/2499
Processing 2413/2499
Processing 2414/2499
Processing 2415/2499
Processing 2416/2499
Processing 2417/2499
Processing 2418/2499
Processing 2419/2499
Processing 2420/2499
Processing 2421/2499
Processing 2422/2499
Processing 2423/2499
Processing 2424/2499
Processing 2425/2499
Processing 2426/2499
Processing 2427/2499
Processing 2428/2499
Processing 2429/2499
Processing 2430/2499
Processing 2431/2499
Processing 2432/2499
Processing 2433/2499
Processing 2434/2499
Processing 2435/2499
Processing 2436/2499
Processing 2437/2499
Processing 2438/2499
Processing 2439/2499
Processing 2440/2499
Processing 2441/2499
Processing 2442/2499
Processing 2443/2499
Processing 2444/2499
Processing 2445/2499
Processing 2446/2499
Processing 2447/2499
Processing 2448/2499
Processing 2449/2499
Processing 24